### Individual Assignment for the class of Statistical and Machine Learning

Code from the class and from the online sources where used in this project.

## 1. Initiation

In [1]:
# install.packages("vita")
# install.packages("dataPreparation")
# install.packages("woeBinning")
# install.packages("Boruta")
# install.packages("xgboost")

In [2]:
Sys.setenv(LANG = "en")

# Data processing library
library(data.table)       # Data manipulation
library(plyr)             # Data manipulation
library(stringr)          # String, text processing
library(vita)             # Quickly check variable importance
library(dataPreparation)  # Data preparation library
library(woeBinning)       # Decision tree–based binning for numerical and categorical variables
library(Boruta)           # Variable selection

# Machine learning library
library(mlr)          # Machine learning framework
library(caret)         # Data processing and machine learning framework
library(MASS)          # LDA
library(randomForest)  # RF
library(gbm)           # Boosting Tree
library(xgboost)       # XGboost

Warning message:
"package 'vita' was built under R version 3.6.3"Warning message:
"package 'dataPreparation' was built under R version 3.6.3"Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:plyr':

    here

The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following object is masked from 'package:base':

    date

Loading required package: Matrix
Loading required package: progress
dataPreparation 0.4.3
Type dataPrepNews() to see new features/changes/bug fixes.
Warning message:
"package 'Boruta' was built under R version 3.6.3"Loading required package: ranger
Warning message:
"package 'ranger' was built under R version 3.6.3"Warning message:
"package 'mlr' was built under R version 3.6.2"Loading required package: ParamHelpers
Warning message:
"package 'ParamHelpers' was built under R version 3.6.2"'mlr' is in maintenance mo

In [3]:
#install.packages('funModeling')

## 2. Data summary and processing

### 2.1. Data summary

#### Read and print out some data

In [4]:
setwd("D:/User/Google Drive/Proyectos/Estudios/Maestría/Courses/15. Statistical & Machine Learning/Individual Assignment/Data")

In [5]:
# Read train (full), test (holdout)
train_full <- read.csv('bank_mkt_train.csv')  # Training dataset
test_holdout <- read.csv('bank_mkt_test.csv')  # Holdout data set without response

In [6]:
print(paste("Number of observations in the train is",nrow(train_full)),quote=FALSE)
print(paste("Number of observations in the train is",nrow(test_holdout)),quote=FALSE)

[1] Number of observations in the train is 7000
[1] Number of observations in the train is 3000


In [7]:
# Print out to check the data type
# Both of the datasets have the same information (one is for the train set and the other one will be used as a test set)
str(train_full)

'data.frame':	7000 obs. of  21 variables:
 $ client_id     : int  2 3 4 5 6 7 8 9 14 15 ...
 $ age           : int  29 39 49 32 29 51 34 52 52 29 ...
 $ job           : Factor w/ 12 levels "admin.","blue-collar",..: 4 11 2 7 1 7 2 8 1 1 ...
 $ marital       : Factor w/ 4 levels "divorced","married",..: 3 2 2 3 3 2 2 2 2 3 ...
 $ education     : Factor w/ 8 levels "basic.4y","basic.6y",..: 4 3 2 7 4 7 1 4 7 7 ...
 $ default       : Factor w/ 2 levels "no","unknown": 1 2 2 1 2 2 1 1 1 1 ...
 $ housing       : Factor w/ 3 levels "no","unknown",..: 1 3 1 3 3 3 3 3 3 3 ...
 $ loan          : Factor w/ 3 levels "no","unknown",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ contact       : Factor w/ 2 levels "cellular","telephone": 2 2 1 1 1 2 1 1 1 1 ...
 $ month         : Factor w/ 10 levels "apr","aug","dec",..: 7 5 8 7 4 5 8 8 8 5 ...
 $ day_of_week   : Factor w/ 5 levels "fri","mon","thu",..: 2 1 4 2 1 4 4 4 3 2 ...
 $ campaign      : int  3 6 2 3 2 1 1 1 3 1 ...
 $ pdays         : int  999 999 999 999 9

In [8]:
head(train_full)

client_id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,subscribe
2,29,housemaid,single,high.school,no,no,no,telephone,may,...,3,999,0,nonexistent,1.1,93.994,-36.4,4.858,5191.0,0
3,39,unemployed,married,basic.9y,unknown,yes,no,telephone,jun,...,6,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,0
4,49,blue-collar,married,basic.6y,unknown,no,no,cellular,nov,...,2,999,0,nonexistent,-0.1,93.200,-42.0,4.153,5195.8,0
5,32,self-employed,single,university.degree,no,yes,no,cellular,may,...,3,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,0
6,29,admin.,single,high.school,unknown,yes,no,cellular,jul,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,0
7,51,self-employed,married,university.degree,unknown,yes,no,telephone,jun,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1,0


#### Correct the variable: campaign

Since campaign includes also the last contact, its value should be reduce by 1.

In [9]:
# Fix the value
train_full[, 'campaign'] <- train_full[, 'campaign'] - 1
test_holdout[, 'campaign'] <- test_holdout[, 'campaign'] - 1
# Quick check
# min(train_full[, 'campaign'])  # Previously = 1
# min(test_holdout[, 'campaign'])  # Previously = 1

#### Check and fix data error (if any)

In [10]:
# Check missing value
apply(is.na(train_fullTh ), 2, sum)

client_id            age            job        marital      education 
             0              0              0              0              0 
       default        housing           loan        contact          month 
             0              0              0              0              0 
   day_of_week       campaign          pdays       previous       poutcome 
             0              0              0              0              0 
  emp.var.rate cons.price.idx  cons.conf.idx      euribor3m    nr.employed 
             0              0              0              0              0 
     subscribe 
             0

#### Check outliers

In [11]:
# install.packages("outliers")
library("outliers")
outlier(train_full$age)


Attaching package: 'outliers'

The following object is masked from 'package:randomForest':

    outlier



[1] 98

In [12]:
library(psych)
describe(train_full)

Warning message:
"package 'psych' was built under R version 3.6.2"
Attaching package: 'psych'

The following object is masked from 'package:outliers':

    outlier

The following object is masked from 'package:randomForest':

    outlier

The following objects are masked from 'package:ggplot2':

    %+%, alpha



,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
client_id,1,7000,5002.60785714,2891.9901535,5018.500,5001.87285714,3725.0325000,2.000,9998.000,9996.000,0.004286773,-1.2005890,34.565893693
age,2,7000,40.34357143,10.6024871,38.000,39.60107143,10.3782000,18.000,98.000,80.000,0.822653018,0.9674292,0.126723959
job*,3,7000,4.74214286,3.5794329,3.000,4.50375000,2.9652000,1.000,12.000,11.000,0.438387413,-1.3953122,0.042782406
marital*,4,7000,2.16942857,0.5996925,2.000,2.21035714,0.0000000,1.000,4.000,3.000,-0.052613827,-0.3175492,0.007167696
education*,5,7000,4.77185714,2.1353225,4.000,4.91214286,2.9652000,1.000,8.000,7.000,-0.257946129,-1.1959371,0.025521986
default*,6,7000,1.20414286,0.4031027,1.000,1.13017857,0.0000000,1.000,2.000,1.000,1.467689986,0.1541360,0.004817998
housing*,7,7000,2.06442857,0.9856770,3.000,2.08053571,0.0000000,1.000,3.000,2.000,-0.129067394,-1.9583801,0.011781094
loan*,8,7000,1.31671429,0.7132775,1.000,1.14589286,0.0000000,1.000,3.000,2.000,1.869634180,1.5857008,0.008525296
contact*,9,7000,1.35971429,0.4799509,1.000,1.32464286,0.0000000,1.000,2.000,1.000,0.584500147,-1.6585965,0.005736510
month*,10,7000,5.24871429,2.3338330,5.000,5.32642857,2.9652000,1.000,10.000,9.000,-0.299013583,-1.0127403,0.027894640


So, when is the skewness too much?  The rule of thumb seems to be:

* If the skewness is between -0.5 and 0.5, the data are fairly symmetrical
* If the skewness is between -1 and – 0.5 or between 0.5 and 1, the data are moderately skewed
* If the skewness is less than -1 or greater than 1, the data are highly skewed


Kurtosis:

* Mesokurtic: This distribution has kurtosis statistic similar to that of the normal distribution. It means that the extreme values of the distribution are similar to that of a normal distribution characteristic. This definition is used so that the standard normal distribution has a kurtosis of three.
* Leptokurtic (Kurtosis > 3): Distribution is longer, tails are fatter. Peak is higher and sharper than Mesokurtic, which means that data are heavy-tailed or profusion of outliers.
Outliers stretch the horizontal axis of the histogram graph, which makes the bulk of the data appear in a narrow (“skinny”) vertical range, thereby giving the “skinniness” of a leptokurtic distribution.
* Platykurtic: (Kurtosis < 3): Distribution is shorter, tails are thinner than the normal distribution. The peak is lower and broader than Mesokurtic, which means that data are light-tailed or lack of outliers.
The reason for this is because the extreme values are less than that of the normal distribution.

We can observe that the following variables that are highly skewed
* default, loan, campaign, pdays, previous, nr.employed, subscribe

We observe that the variable with a high kurtosis are the following:
* campaign, pdays, previous	

#### Split train (full) data into train, valid, test (60:20:20)

In [13]:
set.seed(1)

train_idx <- caret::createDataPartition(y=train_full[, 'subscribe'], p=.6, list=F)
train <- train_full[train_idx, ]  # Train 60%
valid_test <- train_full[-train_idx, ]  # Valid + Test 40%

valid_idx <- caret::createDataPartition(y=valid_test[, 'subscribe'], p=.5, list=F)
valid <- valid_test[valid_idx, ]  # Valid 20%
test <- valid_test[-valid_idx, ]  # Test 20%

#### Check the target variable class distribution

In [14]:
# By number
table(train$subscribe)
table(valid$subscribe)
table(test$subscribe)


   0    1 
3721  479 


   0    1 
1247  153 


   0    1 
1210  190 

In [15]:
# By percentage
table(train$subscribe) / nrow(train)
table(valid$subscribe) / nrow(valid)
table(test$subscribe) / nrow(test)


        0         1 
0.8859524 0.1140476 


        0         1 
0.8907143 0.1092857 


        0         1 
0.8642857 0.1357143 

#### Simply check which variables are potentially important

Note: Running this permutation algorithm may take some time.

In [16]:
# PIMP-Algorithm For The Permutation Variable Importance Measure
# https://cran.r-project.org/web/packages/vita/vita.pdf
X <- train[, 2:(ncol(train)-1)]
y <- as.factor(train[, 'subscribe'])
rf_model <- randomForest(X, y, mtry=3, ntree=100, importance=T, seed=1)
pimp_varImp <- PIMP(X, y, rf_model, S=10, parallel=F, seed=123)

In [17]:
# Print out top most important variables
pimp_varImp$VarImp[order(pimp_varImp$VarImp[, 1], decreasing=T), ]

month      euribor3m   emp.var.rate    nr.employed cons.price.idx 
  4.390190e-02   3.533603e-02   2.769730e-02   2.691444e-02   2.533161e-02 
 cons.conf.idx            job            age      education       poutcome 
  2.417736e-02   7.023914e-03   5.379773e-03   3.310956e-03   3.137317e-03 
   day_of_week        contact       previous        marital          pdays 
  1.769988e-03   1.645544e-03   1.620392e-03   1.248047e-03   9.994050e-04 
       default           loan        housing       campaign 
  4.873672e-04  -1.735964e-05  -2.765152e-04  -6.482756e-04

### 2.2. Feature engineering

Hints:
- Focus on the most important variables.
- Create some framework for searching the new variables.

#### Add variable: month_spring

In [18]:
# Add new variable to train and test (holdout)
# Train, valid, test
train[, 'month_spring'] <- as.logical(train$month %in% c('mar', 'apr', 'may'))
valid[, 'month_spring'] <- as.logical(valid$month %in% c('mar', 'apr', 'may'))
test[, 'month_spring'] <- as.logical(test$month %in% c('mar', 'apr', 'may'))
# Test (holdout)
test_holdout[, 'month_spring'] <- as.logical(test_holdout$month %in% c('mar', 'apr', 'may'))

#### Add variable: month_summer

In [19]:
# Add new variable to train and test (holdout)
# Train, valid, test
train[, 'month_summer'] <- as.logical(train$month %in% c('jun', 'jul', 'aug'))
valid[, 'month_summer'] <- as.logical(valid$month %in% c('jun', 'jul', 'aug'))
test[, 'month_summer'] <- as.logical(test$month %in% c('jun', 'jul', 'aug'))
# Test (holdout)
test_holdout[, 'month_summer'] <- as.logical(test_holdout$month %in% c('jun', 'jul', 'aug'))

#### Add variable: month_autumn

In [20]:
# Add new variable to train and test (holdout)
# Train, valid, test
train[, 'month_autumn'] <- as.logical(train$month %in% c('sep', 'oct', 'nov'))
valid[, 'month_autumn'] <- as.logical(valid$month %in% c('sep', 'oct', 'nov'))
test[, 'month_autumn'] <- as.logical(test$month %in% c('sep', 'oct', 'nov'))
# Test (holdout)
test_holdout[, 'month_autumn'] <- as.logical(test_holdout$month %in% c('sep', 'oct', 'nov'))

#### Add variable: month_winter

In [21]:
# Add new variable to train and test (holdout)
# Train, valid, test
train[, 'month_winter'] <- as.logical(train$month %in% c('dec', 'jan', 'feb'))
valid[, 'month_winter'] <- as.logical(valid$month %in% c('dec', 'jan', 'feb'))
test[, 'month_winter'] <- as.logical(test$month %in% c('dec', 'jan', 'feb'))
# Test (holdout)
test_holdout[, 'month_winter'] <- as.logical(test_holdout$month %in% c('dec', 'jan', 'feb'))

#### Add variable: age > mean(age)

In [22]:
# Add new variable to train and test (holdout)
# Train, valid, test
train[, 'age_ge_mean'] <- as.logical(train$age > mean(train$age))
valid[, 'age_ge_mean'] <- as.logical(valid$age > mean(valid$age))
test[, 'age_ge_mean'] <- as.logical(test$age > mean(test$age))
# Test (holdout)
test_holdout[, 'age_ge_mean'] <- as.logical(test_holdout$age > mean(train$age))

#### Add variable: pdays_999

In [23]:
# Add new variable to train and test (holdout)
# pdays == 999 is a special value
# Train, valid, test
train[, 'pdays_999'] <- as.logical(train$pdays == 999)
valid[, 'pdays_999'] <- as.logical(valid$pdays == 999)
test[, 'pdays_999'] <- as.logical(test$pdays == 999)
# Test (holdout)
test_holdout[, 'pdays_999'] <- as.logical(test_holdout$pdays == 999)

### 2.3. Processing data

#### 2.3.1. Value transformation

- Categorical variables: remapping
- Continuous variables: discretization

Reference:  

Coussement, K., Lessmann, S., & Verstraeten, G. (2017). A comparative analysis of data preparation algorithms for customer churn prediction: A case study in the telecommunication industry. Decision Support Systems, 95, 27-36.

#### Get the list of categorical, boolean and numerical variables

In [24]:
# Get the IV and DV list name
# Dependent variable (DV)
dv_list <- c('subscribe')
# Independent variable (IV)
iv_list <- setdiff(colnames(train), dv_list)  # Exclude the target variable
iv_list <- setdiff(iv_list, 'client_id')  # Exclude the client_id

In [25]:
# Pick out categorical, boolean and numerical variable
iv_cat_list <- c()  # List to store categorical variable
iv_bool_list <- c()  # List to store boolean variable
iv_num_list <- c()  # List to store numerical variable
for (v in iv_list) {
    if (class(train[, v]) == 'factor') {  # Factor == categorical variable
        iv_cat_list <- c(iv_cat_list, v)
    } else if (class(train[, v]) == 'logical') {  # Logical == boolean variable
        iv_bool_list <- c(iv_bool_list, v)
    } else {  # Non-factor + Non-logical == numerical variable
        iv_num_list <- c(iv_num_list, v)
    }
}

#### Grouping (or remapping) categorical variables - Decision tree–based remapping

Reference:  

Package ‘woeBinning’: https://cran.r-project.org/web/packages/woeBinning/woeBinning.pdf

Test the variable remmaping on a categorical variable.

In [26]:
# Grouping 12 categories in the variable job onto 3 groups using WOE
binning_cat <- woe.binning(train, 'subscribe', 'job')
binning_cat

,Group.2,Group.1,woe,iv.total.final,1,0,col.perc.a,col.perc.b,iv.bins
2,management + technician + misc. level neg. + blue-collar + services,blue-collar,-26.98343,0.1237265,235,2391,0.4906054,0.6425692,0.041005042
3,management + technician + misc. level neg. + blue-collar + services,entrepreneur,-26.98343,0.1237265,235,2391,0.4906054,0.6425692,0.041005042
4,management + technician + misc. level neg. + blue-collar + services,housemaid,-26.98343,0.1237265,235,2391,0.4906054,0.6425692,0.041005042
5,management + technician + misc. level neg. + blue-collar + services,management,-26.98343,0.1237265,235,2391,0.4906054,0.6425692,0.041005042
6,management + technician + misc. level neg. + blue-collar + services,services,-26.98343,0.1237265,235,2391,0.4906054,0.6425692,0.041005042
7,management + technician + misc. level neg. + blue-collar + services,technician,-26.98343,0.1237265,235,2391,0.4906054,0.6425692,0.041005042
8,management + technician + misc. level neg. + blue-collar + services,unknown,-26.98343,0.1237265,235,2391,0.4906054,0.6425692,0.041005042
1,admin.,admin.,15.07074,0.1237265,135,902,0.2818372,0.2424080,0.005942273
9,misc. level pos.,retired,68.22718,0.1237265,109,428,0.2275574,0.1150228,0.076779164
10,misc. level pos.,self-employed,68.22718,0.1237265,109,428,0.2275574,0.1150228,0.076779164


In [27]:
# Apply the binning to data
tmp <- woe.binning.deploy(train, binning_cat, add.woe.or.dum.var='woe')
head(tmp[, c('job', 'job.binned', 'woe.job.binned')])

,job,job.binned,woe.job.binned
1,housemaid,management + technician + misc. level neg. + blue-collar + services,-26.98343
4,self-employed,misc. level pos.,68.22718
7,blue-collar,management + technician + misc. level neg. + blue-collar + services,-26.98343
14,technician,management + technician + misc. level neg. + blue-collar + services,-26.98343
15,management,management + technician + misc. level neg. + blue-collar + services,-26.98343
16,admin.,admin.,15.07074


Apply the variable remmaping for all categorical variables.

In [28]:
# Loop through all categorical variables
for (v in iv_cat_list) {
    
    # Remapping categorical variable on train data
    binning_cat <- woe.binning(train, 'subscribe', v)
    
    # Apply the binning to the train, valid and test data
    train <- woe.binning.deploy(train, binning_cat, add.woe.or.dum.var='woe')
    valid <- woe.binning.deploy(valid, binning_cat, add.woe.or.dum.var='woe')
    test <- woe.binning.deploy(test, binning_cat, add.woe.or.dum.var='woe')
    
    # Apply the binning to the test (holdout) data
    test_holdout <- woe.binning.deploy(test_holdout, binning_cat, add.woe.or.dum.var='woe')
}

#### Grouping (or discretizing) numerical variables - Decision tree–based discretization

Test the variable discretizing on a numerical variable.

In [29]:
# Grouping the variable age onto 4 groups using WOE
binning_num <- woe.binning(train, 'subscribe', 'age')
binning_num

,woe,cutpoints.final,cutpoints.final[-1],iv.total.final,1,0,col.perc.a,col.perc.b,iv.bins
"(-Inf,26]",62.641510,-Inf,26,0.1257925,46,191,0.0960334,0.05133029,0.028002706
"(26,38]",1.589851,26,38,0.1257925,223,1705,0.4655532,0.45821016,0.000116744
"(38,50]",-53.033993,38,50,0.1257925,89,1175,0.1858038,0.31577533,0.068929114
"(50, Inf]",36.886531,50,Inf,0.1257925,121,650,0.2526096,0.17468422,0.028743969
Missing,NA,Inf,Missing,0.1257925,0,0,0.0000000,0.00000000,NA


In [30]:
# Apply the binning to data
tmp <- woe.binning.deploy(train, binning_num, add.woe.or.dum.var='woe')
head(tmp[, c('age', 'age.binned', 'woe.age.binned')])

,age,age.binned,woe.age.binned
1,29,"(26,38]",1.589851
4,32,"(26,38]",1.589851
7,34,"(26,38]",1.589851
14,39,"(38,50]",-53.033993
15,39,"(38,50]",-53.033993
16,33,"(26,38]",1.589851


Apply the variable discretizing for all numerical variables.

In [31]:
# Loop through all numerical variables
for (v in iv_num_list) {
    
    # Discretizing numerical variable on train data
    binning_num <- woe.binning(train, 'subscribe', v)
    
    # Apply the binning to the train, valid and test data
    train <- woe.binning.deploy(train, binning_num, add.woe.or.dum.var='woe')
    valid <- woe.binning.deploy(valid, binning_num, add.woe.or.dum.var='woe')
    test <- woe.binning.deploy(test, binning_num, add.woe.or.dum.var='woe')
    
    # Apply the binning to the test (holdout) data
    test_holdout <- woe.binning.deploy(test_holdout, binning_num, add.woe.or.dum.var='woe')
}

#### Grouping (or discretizing) numerical variables - Equal frequency discretization

Reference:  

Tutorial to prepare train and test set using dataPreparation: https://cran.r-project.org/web/packages/dataPreparation/vignettes/train_test_prep.html

Test the variable discretizing on a numerical variable.

In [32]:
# Build the discretization
bins <- build_bins(dataSet=train, cols="age", n_bins=5, type="equal_freq", verbose=F)

# Print out to check
bins

$age
[1] -Inf   31   35   41   50  Inf

In [33]:
# Apply to the data
tmp <- fastDiscretization(dataSet=train, bins=bins, verbose=F)
setDF(tmp); setDF(train)  # Convert data.table to data.frame
head(tmp[, 'age'])

[1] ]-Inf, 31[ [31, 35[   [31, 35[   [35, 41[   [35, 41[   [31, 35[  
Levels: [31, 35[ [35, 41[ [41, 50[ [50, +Inf[ ]-Inf, 31[

Apply the variable discretizing for all numerical variables.

In [34]:
# Loop through all numerical variables
for (v in iv_num_list) {
    
    # Discretizing numerical variable on train data, n_bins=5
    bins <- build_bins(dataSet=train, cols=v, n_bins=5, type="equal_freq", verbose=F)
    
    # Apply the binning to the train, valid and test data
    tmp <- fastDiscretization(dataSet=train, bins=bins, verbose=F)
    setDF(tmp); setDF(train)  # Convert data.table to data.frame
    train[, paste0(v, '_freq_bin')] <- tmp[, v]  # Add new variable
    
    tmp <- fastDiscretization(dataSet=valid, bins=bins, verbose=F)
    setDF(tmp); setDF(valid)  # Convert data.table to data.frame
    valid[, paste0(v, '_freq_bin')] <- tmp[, v]  # Add new variable
    
    tmp <- fastDiscretization(dataSet=test, bins=bins, verbose=F)
    setDF(tmp); setDF(test)  # Convert data.table to data.frame
    test[, paste0(v, '_freq_bin')] <- tmp[, v]  # Add new variable
    
    # Apply the binning to the test (holdout) data
    tmp <- fastDiscretization(dataSet=test_holdout, bins=bins, verbose=F)
    setDF(tmp); setDF(test_holdout)  # Convert data.table to data.frame
    test_holdout[, paste0(v, '_freq_bin')] <- tmp[, v]  # Add new variable
}

#### Grouping (or discretizing) numerical variables - Equal width discretization

Test the variable discretizing on a numerical variable.

In [35]:
# Build the discretization
bins <- build_bins(dataSet=train, cols="age", n_bins=5, type="equal_width", verbose=F)

# Print out to check
bins

$age
[1] 19.0 32.8 46.6 60.4 74.2 88.0

In [36]:
# Apply to the data
tmp <- fastDiscretization(dataSet=train, bins=bins, verbose=F)
setDF(tmp); setDF(train)  # Convert data.table to data.frame
head(tmp[, 'age'])

[1] [19, 32.8[   [19, 32.8[   [32.8, 46.6[ [32.8, 46.6[ [32.8, 46.6[
[6] [32.8, 46.6[
Levels: [19, 32.8[ [32.8, 46.6[ [46.6, 60.4[ [60.4, 74.2[ [74.2, 88]

Apply the variable discretizing for all numerical variables.

In [37]:
# Loop through all numerical variables
for (v in iv_num_list) {
    
    # Discretizing numerical variable on train data, n_bins=5
    bins <- build_bins(dataSet=train, cols=v, n_bins=5, type="equal_width", verbose=F)
    
    # Apply the binning to the train, valid and test data
    tmp <- fastDiscretization(dataSet=train, bins=bins, verbose=F)
    setDF(tmp); setDF(train)  # Convert data.table to data.frame
    train[, paste0(v, '_width_bin')] <- tmp[, v]  # Add new variable
    
    tmp <- fastDiscretization(dataSet=valid, bins=bins, verbose=F)
    setDF(tmp); setDF(valid)  # Convert data.table to data.frame
    valid[, paste0(v, '_width_bin')] <- tmp[, v]  # Add new variable
    
    tmp <- fastDiscretization(dataSet=test, bins=bins, verbose=F)
    setDF(tmp); setDF(test)  # Convert data.table to data.frame
    test[, paste0(v, '_width_bin')] <- tmp[, v]  # Add new variable
    
    # Apply the binning to the test (holdout) data
    tmp <- fastDiscretization(dataSet=test_holdout, bins=bins, verbose=F)
    setDF(tmp); setDF(test_holdout)  # Convert data.table to data.frame
    test_holdout[, paste0(v, '_width_bin')] <- tmp[, v]  # Add new variable
}

#### 2.3.2. Value representation

- Dummy coding
- Incidence replacement
- Weight of evidence (WoE conversion)

Reference:  

Coussement, K., Lessmann, S., & Verstraeten, G. (2017). A comparative analysis of data preparation algorithms for customer churn prediction: A case study in the telecommunication industry. Decision Support Systems, 95, 27-36.

All about Categorical Variable Encoding: https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02

#### Get the updated list of categorical, boolean and numerical variables

In [38]:
# Get the IV and DV list name
# Dependent variable (DV)
dv_list <- c('subscribe')
# Independent variable (IV)
iv_list <- setdiff(colnames(train), dv_list)  # Exclude the target variable
iv_list <- setdiff(iv_list, 'client_id')  # Exclude the client_id

In [39]:
# Pick out categorical, boolean and numerical variable
iv_cat_list <- c()  # List to store categorical variable
iv_bool_list <- c()  # List to store boolean variable
iv_num_list <- c()  # List to store numerical variable
for (v in iv_list) {
    if (class(train[, v]) == 'factor') {  # Factor == categorical variable
        iv_cat_list <- c(iv_cat_list, v)
    } else if (class(train[, v]) == 'logical') {  # Logical == boolean variable
        iv_bool_list <- c(iv_bool_list, v)
    } else {  # Non-factor + Non-logical == numerical variable
        iv_num_list <- c(iv_num_list, v)
    }
}

#### Convert categorical variables to dummy

Test the variable representation on a categorical variable.

In [40]:
# Build the dummy encoding
encoding <- build_encoding(dataSet=train, cols="job", verbose=F)

In [41]:
# Transform the categorical variable
tmp <- one_hot_encoder(dataSet=train, encoding=encoding, type='logical', drop=F, verbose=F)
setDF(tmp)
tmp <- tmp[, -ncol(tmp)]
head(tmp[, 84:ncol(tmp)])

job.admin.,job.blue.collar,job.entrepreneur,job.housemaid,job.management,job.retired,job.self.employed,job.services,job.student,job.technician,job.unemployed
FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE
FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


Apply the variable representation for all categorical variables.

In [42]:
# Loop through all categorical variables
for (v in iv_cat_list) {
    
    # Representing categorical variable on train data
    encoding <- build_encoding(dataSet=train, cols=v, verbose=F)
    
    # Apply the binning to the train, valid and test data
    train <- one_hot_encoder(dataSet=train, encoding=encoding, type='logical', drop=F, verbose=F)
    setDF(train)
    train <- train[, -ncol(train)]  # Drop the last dummy column
    
    valid <- one_hot_encoder(dataSet=valid, encoding=encoding, type='logical', drop=F, verbose=F)
    setDF(valid)
    valid <- valid[, -ncol(valid)]  # Drop the last dummy column
    
    test <- one_hot_encoder(dataSet=test, encoding=encoding, type='logical', drop=F, verbose=F)
    setDF(test)
    test <- test[, -ncol(test)]  # Drop the last dummy column
    
    # Apply the binning to the test (holdout) data
    test_holdout <- one_hot_encoder(dataSet=test_holdout, encoding=encoding, type='logical', drop=F, verbose=F)
    setDF(test_holdout)
    test_holdout <- test_holdout[, -ncol(test_holdout)]  # Drop the last dummy column
}

#### Represent categorical variables using incidence of target variable

Test the variable representation on a categorical variable.

In [43]:
# Find the incidence rates per category of a variable
tb <- table(train$job, train$subscribe)
incidence_map <- data.frame('v1'=rownames(tb), 'v2'=tb[, '1'] / (tb[, '0'] + tb[, '1']))
colnames(incidence_map) <- c('job', 'job_incidence')
incidence_map

,job,job_incidence
admin.,admin.,0.13018322
blue-collar,blue-collar,0.08278146
entrepreneur,entrepreneur,0.10493827
housemaid,housemaid,0.08256881
management,management,0.11006289
retired,retired,0.24102564
self-employed,self-employed,0.13548387
services,services,0.07616708
student,student,0.29347826
technician,technician,0.09571429


In [44]:
# Convert the categories with incidences
tmp <- plyr::join(x=train, y=incidence_map, by='job', type="left", match="all")  # Left join
head(tmp[, c('job', 'job_incidence')])

job,job_incidence
housemaid,0.08256881
self-employed,0.13548387
blue-collar,0.08278146
technician,0.09571429
management,0.11006289
admin.,0.13018322


Apply the variable representation for all categorical variables.

In [45]:
# Loop through all categorical variables
for (v in iv_cat_list) {
    
    # Find the incidence rates per category of a variable
    tb <- table(train[, v], train[, 'subscribe'])
    incidence_map <- data.frame('v1'=rownames(tb), 'v2'=tb[, '1'] / (tb[, '0'] + tb[, '1']))
    colnames(incidence_map) <- c(v, paste0(v, '_incidence'))  # Rename the columns to join
    
    # Apply the variable representation to the train, valid and test data
    train <- plyr::join(x=train, y=incidence_map, by=v, type="left", match="all")
    valid <- plyr::join(x=valid, y=incidence_map, by=v, type="left", match="all")
    test <- plyr::join(x=test, y=incidence_map, by=v, type="left", match="all")
    
    # Apply the binning to the test (holdout) data
    test_holdout <- plyr::join(x=test_holdout, y=incidence_map, by=v, type="left", match="all")
}

#### Represent categorical variables using weight-of-evidence conversion

Test the variable representation on a categorical variable.

In [46]:
# Find the WOE per category of a variable
tb <- table(train$job, train$subscribe)
woe_map <- data.frame('v1'=rownames(tb), 'v2'=log(tb[, '1'] / tb[, '0']))
colnames(woe_map) <- c('job', 'job_woe')
woe_map

,job,job_woe
admin.,admin.,-1.8993397
blue-collar,blue-collar,-2.4051417
entrepreneur,entrepreneur,-2.1435204
housemaid,housemaid,-2.4079456
management,management,-2.0900988
retired,retired,-1.1470647
self-employed,self-employed,-1.8533174
services,services,-2.4956019
student,student,-0.8785504
technician,technician,-2.2457778


In [47]:
# Convert the categories with WOE
tmp <- plyr::join(x=train, y=woe_map, by='job', type="left", match="all")  # Left join
head(tmp[, c('job', 'job_woe')])

job,job_woe
housemaid,-2.407946
self-employed,-1.853317
blue-collar,-2.405142
technician,-2.245778
management,-2.090099
admin.,-1.899340


Apply the variable representation for all categorical variables.

In [48]:
# Loop through all categorical variables
for (v in iv_cat_list) {
    
    # Find the incidence rates per category of a variable
    tb <- table(train[, v], train[, 'subscribe'])
    woe_map <- data.frame('v1'=rownames(tb), 'v2'=log(tb[, '1'] / tb[, '0']))
    colnames(woe_map) <- c(v, paste0(v, '_woe'))  # Rename the columns to join
    
    # Apply the variable representation to the train, valid and test data
    train <- plyr::join(x=train, y=woe_map, by=v, type="left", match="all")
    valid <- plyr::join(x=valid, y=woe_map, by=v, type="left", match="all")
    test <- plyr::join(x=test, y=woe_map, by=v, type="left", match="all")
    
    # Apply the binning to the test (holdout) data
    test_holdout <- plyr::join(x=test_holdout, y=woe_map, by=v, type="left", match="all")
}

#### 2.3.3. Others variable transformations

#### Log transformation numerical variable

In [49]:
# Transform the variable age on train and test (holdout)
# Train, valid, test
train[, 'age_log'] <- log(train[, 'age'])
valid[, 'age_log'] <- log(valid[, 'age'])
test[, 'age_log'] <- log(test[, 'age'])
# Test (holdout)
test_holdout[, 'age_log'] <- log(test_holdout[, 'age'])

#### Standardize numerical variable

In [50]:
# Standardize the variable age on train and test (holdout)
# Train, valid, test
train[, 'age_scaled'] <- scale(train[, 'age'], center=T, scale=T)  # sd = 1, mean = 0
valid[, 'age_scaled'] <- scale(valid[, 'age'], center=T, scale=T)  # sd = 1, mean = 0
test[, 'age_scaled'] <- scale(test[, 'age'], center=T, scale=T)  # sd = 1, mean = 0
# Test (holdout)
test_holdout[, 'age_scaled'] <- scale(test_holdout[, 'age'], center=T, scale=T)  # sd = 1, mean = 0

### 2.4. Variable selection

Reference:  

Verbeke, W., Dejaeger, K., Martens, D., Hur, J., & Baesens, B. (2012). New insights into churn prediction in the telecommunication sector: A profit driven data mining approach. European Journal of Operational Research, 218(1), 211-229.

Boruta: https://www.datacamp.com/community/tutorials/feature-selection-R-boruta

#### Get the updated list of categorical, boolean and numerical variables

In [51]:
# Get the IV and DV list name
# Dependent variable (DV)
dv_list <- c('subscribe')
# Independent variable (IV)
iv_list <- setdiff(colnames(train), dv_list)  # Exclude the target variable
iv_list <- setdiff(iv_list, 'client_id')  # Exclude the client_id

In [52]:
# Pick out categorical, boolean and numerical variable
iv_cat_list <- c()  # List to store categorical variable
iv_bool_list <- c()  # List to store boolean variable
iv_num_list <- c()  # List to store numerical variable
for (v in iv_list) {
    if (class(train[, v]) == 'factor') {  # Factor == categorical variable
        iv_cat_list <- c(iv_cat_list, v)
    } else if (class(train[, v]) == 'logical') {  # Logical == boolean variable
        iv_bool_list <- c(iv_bool_list, v)
    } else {  # Non-factor + Non-logical == numerical variable
        iv_num_list <- c(iv_num_list, v)
    }
}

#### 2.4.1. Variable correcting and filtering

#### Check and correct +/-Inf values (if any)

In [53]:
# Check missing value
# Train, valid, test
sum(apply(sapply(train, is.infinite), 2, sum))
sum(apply(sapply(valid, is.infinite), 2, sum))
sum(apply(sapply(test, is.infinite), 2, sum))
# Test (holdout)
sum(apply(sapply(test_holdout, is.infinite), 2, sum))

[1] 20

[1] 3

[1] 2

[1] 18

In [54]:
# Impute +/-Inf value by NA
# Train, valid, test
train[sapply(train, is.infinite)] <- NA
valid[sapply(valid, is.infinite)] <- NA
test[sapply(test, is.infinite)] <- NA
# Test (holdout)
test_holdout[sapply(test_holdout, is.infinite)] <- NA

#### Check and correct missing values (if any)

In [55]:
# Check missing value
# Train, valid, test
sum(apply(is.na(train), 2, sum))
sum(apply(is.na(valid), 2, sum))
sum(apply(is.na(test), 2, sum))
# Test (holdout)
sum(apply(is.na(test_holdout), 2, sum))

[1] 20

[1] 3

[1] 2

[1] 20

In [56]:
# Impute missing value in numerical variable by mean
for (v in iv_num_list) {
    # Train, valid, test
    train[is.na(train[, v]), v] <- mean(train[, v], na.rm=T)
    valid[is.na(valid[, v]), v] <- mean(valid[, v], na.rm=T)
    test[is.na(test[, v]), v] <- mean(test[, v], na.rm=T)
    
    # Test (holdout)
    test_holdout[is.na(test_holdout[, v]), v] <- mean(test_holdout[, v], na.rm=T)
}

#### Drop categorical variables (all were processed)

In [57]:
for (v in iv_cat_list) {
    # Train, valid, test
    train[, v] <- NULL
    valid[, v] <- NULL
    test[, v] <- NULL
    
    # Test (holdout)
    test_holdout[, v] <- NULL
}

#### Convert boolean variable to numerical

In [58]:
# Convert boolean to int
for (v in iv_bool_list) {
    # Train, valid, test
    train[, v] <- as.integer(train[, v])
    valid[, v] <- as.integer(valid[, v])
    test[, v] <- as.integer(test[, v])
    
    # Test (holdout)
    test_holdout[, v] <- as.integer(test_holdout[, v])
}

#### Drop constant variable (i.e. variance=0)

In [59]:
# Find the constant variable
var_list <- c()
for (v in c(iv_num_list, iv_bool_list)) {
    var_list <- c(var_list, var(train[, v], na.rm=T))
}
constant_var <- c(iv_num_list, iv_bool_list)[var_list == 0]
constant_var

[1] "woe.pdays.binned"            "pdays.binned_incidence"     
[3] "previous_freq_bin_incidence" "pdays.binned_woe"           
[5] "previous_freq_bin_woe"       "pdays.binned...Inf.999."    
[7] "pdays.binned..999. Inf."

In [60]:
# Drop the constant variable
for (v in constant_var) {
    # Train, valid, test
    train[, v] <- NULL
    valid[, v] <- NULL
    test[, v] <- NULL
    
    # Test (holdout)
    test_holdout[, v] <- NULL
}

#### 2.4.2. Variable selection: Fisher Score

In [61]:
FisherScore <- function(basetable, depvar, IV_list) {
  "
  This function calculate the Fisher score of a variable.
  
  Ref:
  ---
  Verbeke, W., Dejaeger, K., Martens, D., Hur, J., & Baesens, B. (2012). New insights into churn prediction in the telecommunication sector: A profit driven data mining approach. European Journal of Operational Research, 218(1), 211-229.
  "
  
  # Get the unique values of dependent variable
  DV <- unique(basetable[, depvar])
  
  IV_FisherScore <- c()
  
  for (v in IV_list) {
    fs <- abs((mean(basetable[which(basetable[, depvar]==DV[1]), v]) - mean(basetable[which(basetable[, depvar]==DV[2]), v]))) /
      sqrt((var(basetable[which(basetable[, depvar]==DV[1]), v]) + var(basetable[which(basetable[, depvar]==DV[2]), v])))
    IV_FisherScore <- c(IV_FisherScore, fs)
  }
  
  return(data.frame(IV=IV_list, fisher_score=IV_FisherScore))
}

varSelectionFisher <- function(basetable, depvar, IV_list, num_select=20) {
  "
  This function will calculate the Fisher score for all IVs and select the best
  top IVs.

  Assumption: all variables of input dataset are converted into numeric type.
  "
  
  fs <- FisherScore(basetable, depvar, IV_list)  # Calculate Fisher Score for all IVs
  num_select <- min(num_select, ncol(basetable))  # Top N IVs to be selected
  return(as.vector(fs[order(fs$fisher_score, decreasing=T), ][1:num_select, 'IV']))
}

In [62]:
# Calculate Fisher Score for all variable
# Get the IV and DV list
dv_list <- c('subscribe')  # DV list
iv_list <- setdiff(names(train), dv_list)  # IV list excluded DV
iv_list <- setdiff(iv_list, 'client_id')  # Excluded the client_id
fs <- FisherScore(train, dv_list, iv_list)
head(fs)

IV,fisher_score
age,0.0364485
campaign,0.1443533
pdays,0.4343708
previous,0.3454339
emp.var.rate,0.6531280
cons.price.idx,0.3030515


In [63]:
# Select top 20 variables according to the Fisher Score
best_fs_var <- varSelectionFisher(train, dv_list, iv_list, num_select=50)
head(best_fs_var, 10)

[1] "nr.employed_freq_bin_woe"         "woe.nr.employed.binned"          
 [3] "nr.employed.binned_woe"           "nr.employed_freq_bin_incidence"  
 [5] "nr.employed.binned_incidence"     "nr.employed_width_bin_woe"       
 [7] "nr.employed"                      "emp.var.rate_width_bin_woe"      
 [9] "euribor3m_freq_bin_woe"           "nr.employed.binned...Inf.5076.2."

In [64]:
# Apply variable selection to the data
# Train
var_select <- names(train)[names(train) %in% best_fs_var]
train_processed <- train[, c('client_id', var_select, 'subscribe')]
# Valid
var_select <- names(valid)[names(valid) %in% best_fs_var]
valid_processed <- valid[, c('client_id', var_select, 'subscribe')]
# Test
var_select <- names(test)[names(test) %in% best_fs_var]
test_processed <- test[, c('client_id', var_select, 'subscribe')]
# Test (holdout)
var_select <- names(test_holdout)[names(test_holdout) %in% best_fs_var]
test_holdout_processed <- test_holdout[, c('client_id', var_select)]

### 2.5. Finalize data processing

In [65]:
# Check if train and test (holdout) have same variables
# Train, valid, test
dim(train_processed)
dim(valid_processed)
dim(test_processed)
# Test (holdout)
dim(test_holdout_processed)

[1] 4200   52

[1] 1400   52

[1] 1400   52

[1] 3000   51

In [66]:
# Rename the data columns
for (v in colnames(train_processed)) {
    
    # Fix the column name
    fix_name <- str_replace_all(v, "[^[:alnum:] ]", "_")
    fix_name <- gsub(' +', '', fix_name) 
    
    # Train, valid, test
    colnames(train_processed)[colnames(train_processed) == v] <- fix_name
    colnames(valid_processed)[colnames(valid_processed) == v] <- fix_name
    colnames(test_processed)[colnames(test_processed) == v] <- fix_name
    
    # Test (holdout)
    colnames(test_holdout_processed)[colnames(test_holdout_processed) == v] <- fix_name
}

In [67]:
# Print out to check
head(train_processed)

client_id,pdays,emp_var_rate,euribor3m,nr_employed,pdays_999,woe_month_binned,woe_emp_var_rate_binned,woe_cons_price_idx_binned,woe_cons_conf_idx_binned,...,euribor3m_binned_woe,nr_employed_binned_woe,emp_var_rate_freq_bin_woe,euribor3m_freq_bin_woe,nr_employed_freq_bin_woe,emp_var_rate_width_bin_woe,cons_conf_idx_width_bin_woe,euribor3m_width_bin_woe,nr_employed_width_bin_woe,subscribe
2,999,1.1,4.858,5191.0,1,-33.39693,-94.56851,-123.93836,9.414245,...,-2.616464,-2.896025,-3.289431,-3.192275,-3.289431,-2.995732,-2.671493,-2.929108,-2.896025,0
5,999,-1.8,1.299,5099.1,1,-33.39693,44.14570,30.17728,-59.006784,...,-2.616464,-1.971674,-1.429056,-2.098587,-1.974348,-1.540039,-1.876394,-1.171029,-1.691018,0
8,999,-0.1,4.153,5195.8,1,-33.39693,44.14570,-33.24684,-59.006784,...,-2.616464,-2.896025,-2.363994,-2.098587,-2.363994,-2.370244,-2.756119,-2.639057,-2.896025,0
21,999,1.4,4.967,5228.1,1,-33.39693,-94.56851,-33.24684,-106.548634,...,-2.616464,-2.896025,-2.876386,-2.833213,-2.876386,-2.995732,-2.671493,-2.929108,-2.896025,0
22,999,1.4,4.964,5228.1,1,-33.39693,-94.56851,-33.24684,-106.548634,...,-2.616464,-2.896025,-2.876386,-2.833213,-2.876386,-2.995732,-2.671493,-2.929108,-2.896025,1
24,999,1.4,4.964,5228.1,1,-33.39693,-94.56851,-33.24684,-106.548634,...,-2.616464,-2.896025,-2.876386,-2.833213,-2.876386,-2.995732,-2.671493,-2.929108,-2.896025,0


## 3. Methodology

### 3.1. Logistic Regression model

In [68]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=5, predict="both")

# Define the model
learner <- makeLearner("classif.logreg", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_processed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc, setAggregation(mlr::auc, train.mean)))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

Resampling: cross-validation
Measures:             auc.train   auc.test    
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"[Resample] iter 1:    0.7942316   0.7488084   
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"[Resample] iter 2:    0.7873690   0.7697810   
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-defi

In [70]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)

# Make prediction on valid data
pred <- predict(best_md, newdata=valid_processed[, -1])
performance(pred, measures=mlr::auc)

# Make prediction on test data
pred <- predict(best_md, newdata=test_processed[, -1])
performance(pred, measures=mlr::auc)

# Make prediction on test (holdout) data
pred <- predict(best_md, newdata=test_holdout_processed[, -1])
pred

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"

auc 
0.8066549

In [73]:
# Output predicted file
output <- data.frame(client_id=test_holdout$client_id, subscribe=pred$data$prob.1)
write.csv(output, 'lr_submission_5.csv', row.names=FALSE)

### 3.2. RandomForest model

In [127]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10)

# Define the model
learner <- makeLearner("classif.randomForest", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_processed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
  makeDiscreteParam('ntree', value=c(100, 250, 500, 750, 1000)),
  makeDiscreteParam('mtry', value=round(sqrt((ncol(train_processed)-1) * c(0.1, 0.25, 0.5, 1, 2, 4))))
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

[Tune] Started tuning learner classif.randomForest for parameter set:
          Type len Def               Constr Req Tunable Trafo
ntree discrete   -   - 100,250,500,750,1000   -    TRUE     -
mtry  discrete   -   -        2,4,5,7,10,14   -    TRUE     -
With control class: TuneControlGrid
Imputation value: -0
[Tune-x] 1: ntree=100; mtry=2
[Tune-y] 1: auc.test.mean=0.7246139; time: 0.2 min
[Tune-x] 2: ntree=250; mtry=2
[Tune-y] 2: auc.test.mean=0.7278561; time: 0.3 min
[Tune-x] 3: ntree=500; mtry=2
[Tune-y] 3: auc.test.mean=0.7352703; time: 0.6 min
[Tune-x] 4: ntree=750; mtry=2
[Tune-y] 4: auc.test.mean=0.7392010; time: 0.7 min
[Tune-x] 5: ntree=1000; mtry=2
[Tune-y] 5: auc.test.mean=0.7401417; time: 1.0 min
[Tune-x] 6: ntree=100; mtry=4
[Tune-y] 6: auc.test.mean=0.7257882; time: 0.1 min
[Tune-x] 7: ntree=250; mtry=4
[Tune-y] 7: auc.test.mean=0.7317396; time: 0.3 min
[Tune-x] 8: ntree=500; mtry=4
[Tune-y] 8: auc.test.mean=0.7379840; time: 0.6 min
[Tune-x] 9: ntree=750; mtry=4
[Tune-y]

In [128]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)

In [129]:
# Make prediction on valid data
pred <- predict(best_md, newdata=valid_processed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.7634506

In [130]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_processed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.736803

In [131]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_holdout_processed[, -1])
pred

Prediction: 3000 observations
predict.type: prob
threshold: 0=0.50,1=0.50
time: 0.09
  prob.0 prob.1 response
1  1.000  0.000        0
2  1.000  0.000        0
3  0.738  0.262        0
4  1.000  0.000        0
5  1.000  0.000        0
6  1.000  0.000        0
... (#rows: 3000, #cols: 3)

In [132]:
# Output predicted file
output <- data.frame(client_id=test_holdout$client_id, subscribe=pred$data$prob.1)
write.csv(output, 'rf_submission_4.csv', row.names=FALSE)

### 3.4. Naive Bayes

In [74]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=5, predict="both")

# Define the model
learner <- makeLearner("classif.naiveBayes", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_processed[, -1], target="subscribe")

# Set hyper parameter tuning (this part changes)
tune_params <- makeParamSet(
)
#     makeNumericLearnerParam(id = "laplace", default = 0, lower = 0)
    # makeNumericLearnerParam(id = "threshold", default = 0.001, lower = 0)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation (this part changes)
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc, setAggregation(mlr::auc, train.mean)))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

Resampling: cross-validation
Measures:             auc.train   auc.test    
[Resample] iter 1:    0.7633015   0.7678741   
[Resample] iter 2:    0.7699187   0.7431950   
[Resample] iter 3:    0.7562929   0.8006747   
[Resample] iter 4:    0.7631708   0.7703333   
[Resample] iter 5:    0.7704162   0.7499913   


Aggregated Result: auc.test.mean=0.7664137,auc.train.mean=0.7646200




In [75]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)

# Make prediction on valid data
pred <- predict(best_md, newdata=valid_processed[, -1])
performance(pred, measures=mlr::auc)

# Make prediction on test data
pred <- predict(best_md, newdata=test_processed[, -1])
performance(pred, measures=mlr::auc)

# Make prediction on test (holdout) data
pred <- predict(best_md, newdata=test_holdout_processed[, -1])
pred

auc 
0.7805531

auc 
0.7928817

Prediction: 3000 observations
predict.type: prob
threshold: 0=0.50,1=0.50
time: 2.61
        prob.0       prob.1 response
1 1.000000e+00 1.350207e-18        0
2 1.000000e+00 3.067820e-22        0
3 2.706201e-57 1.000000e+00        1
4 1.000000e+00 3.283557e-22        0
5 1.000000e+00 2.899585e-22        0
6 1.000000e+00 3.276097e-22        0
... (#rows: 3000, #cols: 3)

In [76]:
# Output predicted file
output <- data.frame(client_id=test_holdout$client_id, subscribe=pred$data$prob.1)
write.csv(output, 'lr_submission_6.csv', row.names=FALSE)

### 3.5. KNN

In [77]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=5, predict="both")

# Define the model
learner <- makeLearner("classif.kknn", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_processed[, -1], target="subscribe")

# Set hyper parameter tuning (this part changes)
tune_params <- makeParamSet(
#     makeNumericLearnerParam(id = "laplace", default = 0, lower = 0)
    # makeNumericLearnerParam(id = "threshold", default = 0.001, lower = 0)
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation (this part changes)
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc, setAggregation(mlr::auc, train.mean)))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

Loading required package: kknn
Warning message:
"package 'kknn' was built under R version 3.6.2"
Attaching package: 'kknn'

The following object is masked from 'package:caret':

    contr.dummy

Resampling: cross-validation
Measures:             auc.train   auc.test    
[Resample] iter 1:    0.7791306   0.6693165   
[Resample] iter 2:    0.7677935   0.7173216   
[Resample] iter 3:    0.7717957   0.7187300   
[Resample] iter 4:    0.7809510   0.6900910   
[Resample] iter 5:    0.7676631   0.7337635   


Aggregated Result: auc.test.mean=0.7058445,auc.train.mean=0.7734668




In [78]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)

# Make prediction on valid data
pred <- predict(best_md, newdata=valid_processed[, -1])
performance(pred, measures=mlr::auc)

# Make prediction on test data
pred <- predict(best_md, newdata=test_processed[, -1])
performance(pred, measures=mlr::auc)

# Make prediction on test (holdout) data
pred <- predict(best_md, newdata=test_holdout_processed[, -1])
pred

auc 
0.7309464

auc 
0.6958177

Prediction: 3000 observations
predict.type: prob
threshold: 0=0.50,1=0.50
time: 0.47
     prob.0    prob.1 response
1 1.0000000 0.0000000        0
2 1.0000000 0.0000000        0
3 0.6135519 0.3864481        0
4 0.8979580 0.1020420        0
5 1.0000000 0.0000000        0
6 1.0000000 0.0000000        0
... (#rows: 3000, #cols: 3)

In [79]:
# Output predicted file
output <- data.frame(client_id=test_holdout$client_id, subscribe=pred$data$prob.1)
write.csv(output, 'lr_submission_KNN.csv', row.names=FALSE)

## 3.5 LDA model

In [81]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=5, predict="both")

# Define the model
learner <- makeLearner("classif.lda", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_processed[, -1], target="subscribe")

# Set hyper parameter tuning (this part changes)
tune_params <- makeParamSet(
#     makeNumericLearnerParam(id = "laplace", default = 0, lower = 0)
    # makeNumericLearnerParam(id = "threshold", default = 0.001, lower = 0)
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation (this part changes)
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc, setAggregation(mlr::auc, train.mean)))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

Resampling: cross-validation
Measures:             auc.train   auc.test    
Warning message in lda.default(x, grouping, ...):
"variables are collinear"[Resample] iter 1:    0.7680493   0.8326427   
Warning message in lda.default(x, grouping, ...):
"variables are collinear"[Resample] iter 2:    0.8003541   0.7268244   
Warning message in lda.default(x, grouping, ...):
"variables are collinear"[Resample] iter 3:    0.7794022   0.8070886   
Warning message in lda.default(x, grouping, ...):
"variables are collinear"[Resample] iter 4:    0.7923676   0.7342282   
Warning message in lda.default(x, grouping, ...):
"variables are collinear"[Resample] iter 5:    0.7867347   0.7679816   


Aggregated Result: auc.test.mean=0.7737531,auc.train.mean=0.7853816




In [82]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)

# Make prediction on valid data
pred <- predict(best_md, newdata=valid_processed[, -1])
performance(pred, measures=mlr::auc)

# Make prediction on test data
pred <- predict(best_md, newdata=test_processed[, -1])
performance(pred, measures=mlr::auc)

# Make prediction on test (holdout) data
pred <- predict(best_md, newdata=test_holdout_processed[, -1])
pred

Warning message in lda.default(x, grouping, ...):
"variables are collinear"

auc 
0.8049331

auc 
0.8097803

Prediction: 3000 observations
predict.type: prob
threshold: 0=0.50,1=0.50
time: 0.02
     prob.0     prob.1 response
1 0.9783962 0.02160380        0
2 0.9807609 0.01923913        0
3 0.2113227 0.78867728        1
4 0.9727311 0.02726890        0
5 0.9820760 0.01792395        0
6 0.9722819 0.02771807        0
... (#rows: 3000, #cols: 3)

In [83]:
# Output predicted file
output <- data.frame(client_id=test_holdout$client_id, subscribe=pred$data$prob.1)
write.csv(output, 'lr_submission_LDA.csv', row.names=FALSE)

In [ ]:
# "classif.naiveBayes","classif.earth", "classif.xgboost", "classif.kknn", "classif.randomForest",
#                 "classif.logreg", "classif.lda", "classif.plsdaCaret", "classif.ctree", "classif.C50", "classif.boosting"